In [14]:
%config IPCompleter.greedy=True

### Tensors

In [1]:
from __future__ import print_function
import torch

x=torch.tensor([[1,1],[-1,-1]])
print(x)
y=torch.tensor([[0,0],[1,1]])
print(y)
print(torch.add(x,y))
print(y.add_(x))
x0 = x.view(4)
print(x0)
x=torch.empty(5,3)
z = torch.randn(4,4,4)
z0 = z.view(-1,8,2)
print(z0.size())
print(torch.cuda.is_available())

tensor([[ 1,  1],
        [-1, -1]])
tensor([[0, 0],
        [1, 1]])
tensor([[1, 1],
        [0, 0]])
tensor([[1, 1],
        [0, 0]])
tensor([ 1,  1, -1, -1])
torch.Size([4, 8, 2])
False


###  AutoGrad

In [22]:
x = torch.rand(2,2,requires_grad=True)
#print(x)
y = x + 2
print(y.grad_fn)
z = y * y * 3
z_mean = z.mean()
print(z)
print(z_mean)
z_mean.backward()
print(x.grad)

tensor([[26.2780, 20.5255],
        [21.8914, 21.6872]], grad_fn=<MulBackward0>)
tensor(22.5955, grad_fn=<MeanBackward1>)
tensor([[4.4394, 3.9235],
        [4.0520, 4.0330]])


In [37]:
x = torch.randn(3,3,requires_grad=True)
print(x)

tensor([[-0.7201,  2.1970, -0.0980],
        [ 0.9116, -0.3588,  0.3907],
        [ 0.1504,  0.7995, -0.0346]], requires_grad=True)
